In [1]:
import argparse
import cv2
import os
import numpy as np
from tqdm import tqdm
import re
import imutils

In [23]:
SCENE = "warehouse_1"

DIR = "/home/maciej/Desktop/pallet_dataset/scenes/"+SCENE+"/"

In [28]:
def equalize_hist(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_yuv[:,:,1] = cv2.equalizeHist(img_yuv[:,:,1])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_HSV2BGR)
    out_dir_name="equalized/"
    return img_output, out_dir_name

def denoise(img):
    img_output=cv2.fastNlMeansDenoisingColored(img, None,30)
    out_dir_name="denoised/"
    return img_output, out_dir_name

In [29]:
filenames = sorted(os.listdir(DIR))
filenames=[f for f in filenames if f not in ["equalized", "info.txt", "denoised"]]
for i in tqdm(range(len(filenames))):
    filename=filenames[i]
    filepath=DIR+filename
    print(filepath)
    img=cv2.imread(filepath)
    img_output, out_dir_name = denoise(img)
    out_path=DIR+out_dir_name+filename
    cv2.imwrite(out_path, img_output)

  0%|          | 0/114 [00:00<?, ?it/s]

/home/maciej/Desktop/pallet_dataset/scenes/warehouse_1/equalized/r_1_0.jpg


  1%|          | 1/114 [00:00<00:50,  2.24it/s]

/home/maciej/Desktop/pallet_dataset/scenes/warehouse_1/equalized/r_1_1.jpg


  2%|▏         | 2/114 [00:00<00:49,  2.25it/s]

...

 98%|█████████▊| 112/114 [00:52<00:00,  2.13it/s]

/home/maciej/Desktop/pallet_dataset/scenes/warehouse_1/equalized/r_1_98.jpg


 99%|█████████▉| 113/114 [00:53<00:00,  2.12it/s]

/home/maciej/Desktop/pallet_dataset/scenes/warehouse_1/equalized/r_1_99.jpg


100%|██████████| 114/114 [00:53<00:00,  2.13it/s]
